In [2]:
data = open('C:/Users/Parth/Desktop/Deep Learning/kafka.txt', 'r').read()

chars = list(set(data)) 
data_size, vocab_size = len(data), len(chars)
print ('data has %d chars, %d unique' % (data_size, vocab_size))

data has 137773 chars, 81 unique


In [3]:
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print (char_to_ix)
print (ix_to_char)

{'n': 0, 'r': 1, '.': 2, 'a': 3, 's': 4, 'x': 5, '8': 6, 'X': 7, 'V': 8, 'c': 9, 'M': 10, 'W': 11, 'K': 12, 'e': 13, ':': 14, '$': 15, '©': 16, 'z': 17, 'o': 18, 'B': 19, 'R': 20, '!': 21, 'f': 22, 'ç': 23, 'J': 24, 'P': 25, 't': 26, 'u': 27, 'E': 28, '3': 29, 'k': 30, '6': 31, 'O': 32, 'i': 33, 'A': 34, 'S': 35, 'p': 36, '1': 37, 'v': 38, 'C': 39, 'T': 40, 'd': 41, '-': 42, '\n': 43, 'b': 44, 'y': 45, '?': 46, 'Y': 47, 'Q': 48, '0': 49, 'F': 50, '@': 51, '2': 52, '*': 53, "'": 54, 'l': 55, 'H': 56, 'I': 57, 'N': 58, 'U': 59, 'h': 60, '(': 61, 'L': 62, '9': 63, 'D': 64, '5': 65, 'q': 66, 'j': 67, '4': 68, 'G': 69, '/': 70, 'w': 71, 'g': 72, ' ': 73, '"': 74, '7': 75, '%': 76, 'm': 77, ')': 78, ',': 79, ';': 80}
{0: 'n', 1: 'r', 2: '.', 3: 'a', 4: 's', 5: 'x', 6: '8', 7: 'X', 8: 'V', 9: 'c', 10: 'M', 11: 'W', 12: 'K', 13: 'e', 14: ':', 15: '$', 16: '©', 17: 'z', 18: 'o', 19: 'B', 20: 'R', 21: '!', 22: 'f', 23: 'ç', 24: 'J', 25: 'P', 26: 't', 27: 'u', 28: 'E', 29: '3', 30: 'k', 31: '6', 

In [5]:
import numpy as np

vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
print (vector_for_char_a.ravel())

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
#hyperparameters
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

In [8]:
#model parameters

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #hidden to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 #hidden to output
bh = np.zeros((hidden_size, 1)) #bias to hidden
by = np.zeros((vocab_size, 1)) #bias to output

In [27]:
def lossFun(inputs, targets, hprev):
#store our inputs, hidden states, outputs, and probability values
    xs, hs, ys, ps, = {}, {}, {}, {} #Empty dicts
    # Each of these are going to be SEQ_LENGTH(Here 25) long dicts i.e. 1 vector per time(seq) step
    # xs will store 1 hot encoded input characters for each of 25 time steps (26, 25 times)
    # hs will store hidden state outputs for 25 time steps (100, 25 times)) plus a -1 indexed initial state
    # to calculate the hidden state at t = 0
    # ys will store targets i.e. expected outputs for 25 times (26, 25 times), unnormalized probabs
    # ps will take the ys and convert them to normalized probab for chars
    # We could have used lists BUT we need an entry with -1 to calc the 0th hidden layer
    # -1 as  a list index would wrap around to the final element
    xs, hs, ys, ps = {}, {}, {}, {}
      #init with previous hidden state
    # Using "=" would create a reference, this creates a whole separate copy
    # We don't want hs[-1] to automatically change if hprev is changed
    hs[-1] = np.copy(hprev)
      #init loss as 0
    loss = 0
    # forward pass                                                                                                                                                                              
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation (we place a 0 vector as the t-th input)                                                                                                                     
        xs[t][inputs[t]] = 1 # Inside that t-th input we use the integer in "inputs" list to  set the correct
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
    # backward pass: compute gradients going backwards    
    #initalize vectors for gradient values for each set of weights 
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
    #output probabilities
        dy = np.copy(ps[t])
    #derive our first gradient
        dy[targets[t]] -= 1 # backprop into y  
    #compute output gradient -  output times hidden states transpose
    #When we apply the transpose weight matrix,  
    #we can think intuitively of this as moving the error backward
    #through the network, giving us some sort of measure of the error 
    #at the output of the lth layer. 
    #output gradient
        dWhy += np.dot(dy, hs[t].T)
    #derivative of output bias
        dby += dy
    #backpropagate!
        dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
        dbh += dhraw #derivative of hidden bias
        dWxh += np.dot(dhraw, xs[t].T) #derivative of input to hidden layer weight
        dWhh += np.dot(dhraw, hs[t-1].T) #derivative of hidden layer to hidden layer weight
        dhnext = np.dot(Whh.T, dhraw) 
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [28]:
#prediction, one full forward pass
def sample(h, seed_ix, n):
    #create vector
    x = np.zeros((vocab_size, 1))
    #customize it for our seed char
    x[seed_ix] = 1
    #list to store generated chars
    ixes = []
    #for as many characters as we want to generate
    for t in range(n):
    #a hidden state at a given time step is a function 
    #of the input at the same time step modified by a weight matrix 
    #added to the hidden state of the previous time step 
    #multiplied by its own hidden state to hidden state matrix.
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    #compute output (unnormalised)
        y = np.dot(Why, h) + by
    ## probabilities for next chars
        p = np.exp(y) / np.sum(np.exp(y))
    #pick one with the highest probability 
        ix = np.random.choice(range(vocab_size), p=p.ravel())
    #create a vector
        x = np.zeros((vocab_size, 1))
    #customize it for the predicted char
        x[ix] = 1
    #add it to the list
        ixes.append(ix)

    txt = ''.join(ix_to_char[ix] for ix in ixes)
    print ('----\n %s \n----' % (txt, ))
hprev = np.zeros((hidden_size,1)) # reset RNN memory  
#predict the 200 next characters given 'a'
sample(hprev,char_to_ix['a'],200)


----
 çN5m't,R/Mc1;
kP$9@;tFuçUhCmlTv':i
nCdfo%UrUSsx)* Ce?-wm6yd/Iu1Ew
t©nyDgKfTdXapKGixUYOjd0Ti©k'@,:Eyç$©7Q$KdTDD,4Vv%xeBk9pbK(WTRRWr8gl3çh;çGlT(@ML?eEe1n8,8uUxw/rLiJ:oSQ@Jc68)©X)RpJb.%f3@@rp:M.4©6
HUs1" 
----


In [29]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print ("inputs", inputs)
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print ("targets", targets)

inputs [32, 0, 13, 73, 77, 18, 1, 0, 33, 0, 72, 79, 73, 71, 60, 13, 0, 73, 69, 1, 13, 72, 18, 1, 73]
targets [0, 13, 73, 77, 18, 1, 0, 33, 0, 72, 79, 73, 71, 60, 13, 0, 73, 69, 1, 13, 72, 18, 1, 73, 35]


In [30]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
 # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
        p = 0 # go from start of data                                                                                                                                                             
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
    if n % 1000 == 0:
        print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
        sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
        
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

    p += seq_length # move data pointer                                                                                                                                                         
    n += 1 # iteration counter

iter 0, loss: 109.861224
----
 
/.aQçlM:-e-.X!?Pçmi/QMztyasH-/ldCieao-ç(j%$@STcsBSç61m45q6D"vPDN03
.jeIKl?w"A!iGntC/)Xa"jumxfF?S(!?)MkGO9:vz'I5E©8moYP;NjOkkJ6(C3NGbM,a3w3W:Nl'uUz09QILTz7'F7ezDVAJgSXHr$jk:/IC'Lp;pT;cu*LF3;çld ;kcDgJ 
----
iter 1000, loss: 84.013698
----
 , amlree veithim he tis nik re cl ane, he fnwiecey fhe fiswke he difs. anth, soin an wate the chiaf he dore ge semd chke ritcive ke he cant mny viscenk le te tor napt thal ils, ee the ind cer hiserout 
----
iter 2000, loss: 66.116330
----
 puld ind sieg d aj here ltoovere his hing, if cghe ale ag he whar his leameare couthemhe that mo he corackithurerer ser aveneay. meimuster hish le'cacimous hiske qus she co mome he whithlathe the ou h 
----
iter 3000, loss: 56.892048
----
 d:cither sand, the there ulson tared sved st und gomling"nbithece hing nody aid ond, pelly aryey onct; ttaklver the covererey; ngnurly y she m" er the snonly oudse therd folwmund roihered arent  kers  
----
iter 4000, loss: 52.256488
----
  hims way